In [1]:
from pl_bolts.models.self_supervised import SimCLR
from pl_bolts.models.self_supervised.simclr import SimCLREvalDataTransform, SimCLRTrainDataTransform

In [2]:
import sys
sys.path.append('/om5/user/smadan/training_scaffold_own/res/')
from models.models import get_model
from loader.loader import get_loader

/om5/user/smadan/training_scaffold_own/res/loader/multi_attribute_loader.py
/om5/user/smadan/training_scaffold_own/res/loader
/om5/user/smadan/training_scaffold_own/res/loader/loader.py
/om5/user/smadan/training_scaffold_own/res/loader


In [3]:
from torchvision import transforms
import torch

In [4]:
transforms_without_crop = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
    ])
transforms_with_crop = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.RandomResizedCrop(224),
    ])

image_transform = {}
# image_transform['train'] = transforms_without_crop
# image_transform['test'] = transforms_without_crop


image_transform['train'] = SimCLRTrainDataTransform()
image_transform['test'] = SimCLREvalDataTransform()

In [5]:
loader_new = get_loader('multi_attribute_loader_file_list_shapenet_lightning')
file_list_root = '/om5/user/smadan/dataset_lists_openmind'
att_path = '/om5/user/smadan/differentiable_graphics_ml/training_models/shapenet_id_to_class_num.p'
shuffles = {'train':True,'val':True,'test':False}

In [6]:
wandb_config = {}
wandb_config['dataset_name'] = 'image_train_v7_shapenet_40'
wandb_config['batch_size'] = 5

In [7]:
file_lists = {}
dsets = {}
dset_loaders = {}
dset_sizes = {}
for phase in ['train','test']:
    file_lists[phase] = "%s/%s_list_%s.txt"%(file_list_root,phase,wandb_config['dataset_name'])
    dsets[phase] = loader_new(file_lists[phase],att_path, image_transform[phase])
    dset_loaders[phase] = torch.utils.data.DataLoader(dsets[phase], batch_size=wandb_config['batch_size'], shuffle = shuffles[phase], num_workers=2,drop_last=True)
    dset_sizes[phase] = len(dsets[phase])

In [8]:
for data in dset_loaders['train']:
    break

In [14]:
model = SimCLR(gpus=1, num_samples = 5, batch_size = 20, dataset = dset_loaders['train'])
# trainer = Trainer(gpus=2)

In [15]:
import pytorch_lightning as pl

In [16]:
trainer = pl.Trainer(gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
# train_dataset = MyDataset(transforms=SimCLRTrainDataTransform())
# val_dataset = MyDataset(transforms=SimCLREvalDataTransform())

# simclr needs a lot of compute!
# model = SimCLR()
# trainer = Trainer(tpu_cores=128)
trainer.fit(
    model,
    dset_loaders['train'],
    dset_loaders['test'],
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name       | Type       | Params
------------------------------------------
0 | encoder    | ResNet     | 25.6 M
1 | projection | Projection | 4.5 M 
------------------------------------------
30.0 M    Trainable params
0         Non-trainable params
30.0 M    Total params
120.078   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/om5/user/smadan/miniconda3/envs/diff_rendering_ml/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/om5/user/smadan/miniconda3/envs/diff_rendering_ml/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]